拟合模型的2个目标：优化（train dataset） +泛化（beyond it)

导数 梯度 链式法则 
# 求导重要---用于优化损失函数 必须选可微的Lossfunction

# 用matplotlib 可视化(作图


# 自动微分
设计好模型，系统会构建一个计算图computational graph？ 来跟踪计算是哪些数据通过哪些操作组合起来产生输出。 
自动微分使系统能够随后反向传播梯度backpropagate
backpropagate：跟踪整个计算图，填充关于每个参数的偏导数
gradient: note that 关于向量x的梯度是向量，并且与x具有相同的形状。

In [2]:
import torch

# 启动启用自动求导机制 开始跟踪该张量

In [3]:
x=torch.arange(4.0,requires_grad=True) 
x

tensor([0., 1., 2., 3.], requires_grad=True)

In [4]:
y=2 * torch.dot(x,x)
y

tensor(28., grad_fn=<MulBackward0>)

调用反向传播函数来自动计算y关于x每个分量的梯度

In [5]:
y.backward()
x.grad
#y对x 的梯度 用指示x来显示

tensor([ 0.,  4.,  8., 12.])

In [6]:
x.grad == 4*x  #验证

tensor([True, True, True, True])

# 在默认情况下，PyTorch会累积梯度，我们需要清除之前的值

In [7]:
x.grad.zero_() 

tensor([0., 0., 0., 0.])

In [8]:
y2 = x.sum()
y2.backward()
x.grad

tensor([1., 1., 1., 1.])

In [9]:
#没懂参数？？？？？？？？？
# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
# 本例只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad


tensor([0., 2., 4., 6.])

# 将计算移出计算图 （直接保留此处的结果为常数

In [12]:
x.grad.zero_()
y=x*x
stayc=y.detach()
z=stayc *x

z.sum().backward() #向量的求导一般求个sum（变标量） 再求导
x.grad == stayc #验证 u为常数


tensor([True, True, True, True])

反向累积 反向传递计算（去想图） 其实就是手算复合导数的顺序

In [13]:
 #即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度。
def f(a):
    b=a*2;
    while b.norm()<1000:
        b=b*2
    if b.sum()>0:
        c=b
    else:
        c=100*b
    return c

t=torch.randn(size=(),requires_grad=True)
e=f(t)
e.backward()
t.grad == e/t


tensor(True)

# 分母布局 分子布局
梯度结果 行数与分母相同 则是分母布局demoninator layout
eg f(x)对向量x求梯度 结构是x维度相同的 列向量

如果结果得行向量（1×m) 则一行 与f对x求导，（f/x)的分子相同 则是分子布局numenator

二者互为转置

# y是标量方程 
对向量导  则是普通函数 梯度为向量
# y是向量方程 
向量y 关于 向量x 的导数是一个矩阵
y(x) y也是列向量
若是分母布局 则结果要和向量x为同 行数确定！ 于是每行 对f展开 列数与f维度相同
若是分子布局 则结果行数==f维度 每行 fm分别对x展开

# 向量只是标量的一种装填表示方式而已

TB：
ay导y 为a转置:
A mv y 对y导 结果是A.T 

xTA A:
xTAx 导 Ax+ATx

uv（都标量） 导x ==u(v导x)+v(u导x)

<u,v> 导x (都是向量）
=u(v导x)+v(u导x) 


亚导数 将函数不可导的点进行补充

# 正向求导耗时（要从头到尾扫一遍） 省空间 
# 反向求导耗空间（要存每一步的导）